In [20]:
import pandas as pd
import numpy as np

!wget --load-cookies cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1So6tIcdxbWj3poMGf_acLaOGHfD_ZzEQ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1So6tIcdxbWj3poMGf_acLaOGHfD_ZzEQ" -O alldata.csv && rm -rf cookies.txt 

Cannot open cookies file ‘cookies.txt’: No such file or directory
--2018-11-18 04:18:29--  https://docs.google.com/uc?export=download&confirm=&id=1So6tIcdxbWj3poMGf_acLaOGHfD_ZzEQ
Resolving docs.google.com (docs.google.com)... 172.217.12.46, 2607:f8b0:4000:815::200e
Connecting to docs.google.com (docs.google.com)|172.217.12.46|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0flrttlk005sof6d6s7779rs6h87gufa/1542513600000/11339478990303284305/*/1So6tIcdxbWj3poMGf_acLaOGHfD_ZzEQ?e=download [following]
--2018-11-18 04:18:30--  https://doc-04-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0flrttlk005sof6d6s7779rs6h87gufa/1542513600000/11339478990303284305/*/1So6tIcdxbWj3poMGf_acLaOGHfD_ZzEQ?e=download
Resolving doc-04-10-docs.googleusercontent.com (doc-04-10-docs.googleusercontent.com)... 172.217.9.129, 2607:f8b0:4000:812::2001


In [24]:
df = pd.read_csv('alldata.csv', header=None)

In [123]:
# Check size of data df.shape
# Separate Class and Data
y = df.iloc[:,-1]
X = df.iloc[:,0:-1]

In [140]:
# Split Train and Independent Test (20% as Test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.20)

In [150]:
# Normalization all feature to 0,1
# Start 2104 features
from sklearn.preprocessing import MinMaxScaler
pre_normalize = MinMaxScaler()
pre_normalize.fit(X_train)
X_train_pre = pre_normalize.transform(X_train)
X_test_pre = pre_normalize.transform(X_test)

In [151]:
# Remove low Variance Features
# Use percentile rank 1 as threshold to remove
# 214 features
from sklearn.feature_selection import VarianceThreshold
threshold = np.percentile(X_train_pre.var(axis=1),1)
pre_var = VarianceThreshold(threshold=threshold)
pre_var.fit(X_train_pre)
X_train_pre = pre_var.transform(X_train_pre)
X_test_pre = pre_var.transform(X_test_pre)

In [153]:
# Remove High Correlated Features
# 167 features
corr_matrix = pd.DataFrame(X_train_pre).corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.99
to_drop = [column for column in upper.columns if any(upper[column] > 0.99)]

X_train_pre = pd.DataFrame(X_train_pre).drop(df.columns[to_drop], axis=1)
X_test_pre = pd.DataFrame(X_test_pre).drop(df.columns[to_drop], axis=1)

In [161]:
# Deep Learning
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

# Change Class to dummy variables (hot encoded)
dummy_train = np_utils.to_categorical(y_train)
dummy_test = np_utils.to_categorical(y_test)

# Design My Deepmodel
model = Sequential()
model.add(Dense(32,input_dim=167,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pre, dummy_train, epochs=200, batch_size=10)

Epoch 1/50
3200/3200 [==============================] - 1s 330us/step - loss: 2.0632 - acc: 0.2284
Epoch 2/50
3200/3200 [==============================] - 1s 266us/step - loss: 1.6799 - acc: 0.3097
Epoch 3/50
3200/3200 [==============================] - 1s 278us/step - loss: 1.4961 - acc: 0.3684
Epoch 4/50
3200/3200 [==============================] - 1s 276us/step - loss: 1.4295 - acc: 0.3841
Epoch 5/50
3200/3200 [==============================] - 1s 274us/step - loss: 1.3500 - acc: 0.4113
Epoch 6/50
3200/3200 [==============================] - 1s 258us/step - loss: 1.3443 - acc: 0.4116
Epoch 7/50
3200/3200 [==============================] - 1s 271us/step - loss: 1.2988 - acc: 0.4347
Epoch 8/50
3200/3200 [==============================] - 1s 263us/step - loss: 1.2711 - acc: 0.4500
Epoch 9/50
3200/3200 [==============================] - 1s 274us/step - loss: 1.2442 - acc: 0.4588
Epoch 10/50
3200/3200 [==============================] - 1s 260us/step - loss: 1.2168 - acc: 0.4850
Epoch 11/

In [162]:
model.fit(X_train_pre, dummy_train, epochs=50, batch_size=10)

Epoch 1/50
3200/3200 [==============================] - 1s 285us/step - loss: 0.7924 - acc: 0.7009
Epoch 2/50
3200/3200 [==============================] - 1s 270us/step - loss: 0.7872 - acc: 0.6966
Epoch 3/50
3200/3200 [==============================] - 1s 261us/step - loss: 0.7615 - acc: 0.7022
Epoch 4/50
3200/3200 [==============================] - 1s 273us/step - loss: 0.7779 - acc: 0.7084
Epoch 5/50
3200/3200 [==============================] - 1s 256us/step - loss: 0.7539 - acc: 0.7150
Epoch 6/50
3200/3200 [==============================] - 1s 272us/step - loss: 0.7527 - acc: 0.7109
Epoch 7/50
3200/3200 [==============================] - 1s 263us/step - loss: 0.7504 - acc: 0.7172
Epoch 8/50
3200/3200 [==============================] - 1s 278us/step - loss: 0.7440 - acc: 0.7206
Epoch 9/50
3200/3200 [==============================] - 1s 274us/step - loss: 0.7413 - acc: 0.7147
Epoch 10/50
3200/3200 [==============================] - 1s 260us/step - loss: 0.7232 - acc: 0.7266
Epoch 11/

In [163]:
model.fit(X_train_pre, dummy_train, epochs=50, batch_size=10)

Epoch 1/50
3200/3200 [==============================] - 1s 268us/step - loss: 0.5923 - acc: 0.7844
Epoch 2/50
3200/3200 [==============================] - 1s 277us/step - loss: 0.6044 - acc: 0.7856
Epoch 3/50
3200/3200 [==============================] - 1s 274us/step - loss: 0.6018 - acc: 0.7831
Epoch 4/50
3200/3200 [==============================] - 1s 266us/step - loss: 0.5944 - acc: 0.7928
Epoch 5/50
3200/3200 [==============================] - 1s 274us/step - loss: 0.5661 - acc: 0.7984
Epoch 6/50
3200/3200 [==============================] - 1s 276us/step - loss: 0.5899 - acc: 0.7959
Epoch 7/50
3200/3200 [==============================] - 1s 274us/step - loss: 0.5790 - acc: 0.8044
Epoch 8/50
3200/3200 [==============================] - 1s 267us/step - loss: 0.5965 - acc: 0.7841
Epoch 9/50
3200/3200 [==============================] - 1s 275us/step - loss: 0.5626 - acc: 0.8012
Epoch 10/50
3200/3200 [==============================] - 1s 270us/step - loss: 0.6036 - acc: 0.7872
Epoch 11/

In [181]:
model.fit(X_train_pre, dummy_train, epochs=300, batch_size=10)

Epoch 1/300
3200/3200 [==============================] - 1s 283us/step - loss: 0.3030 - acc: 0.9078
Epoch 2/300
3200/3200 [==============================] - 1s 281us/step - loss: 0.2801 - acc: 0.9225
Epoch 3/300
3200/3200 [==============================] - 1s 277us/step - loss: 0.2972 - acc: 0.9122
Epoch 4/300
3200/3200 [==============================] - 1s 259us/step - loss: 0.2478 - acc: 0.9319
Epoch 5/300
3200/3200 [==============================] - 1s 268us/step - loss: 0.2684 - acc: 0.9184
Epoch 6/300
3200/3200 [==============================] - 1s 259us/step - loss: 0.3500 - acc: 0.8956
Epoch 7/300
3200/3200 [==============================] - 1s 270us/step - loss: 0.3074 - acc: 0.9069
Epoch 8/300
3200/3200 [==============================] - 1s 264us/step - loss: 0.2591 - acc: 0.9269
Epoch 9/300
3200/3200 [==============================] - 1s 271us/step - loss: 0.2574 - acc: 0.9287
Epoch 10/300
3200/3200 [==============================] - 1s 274us/step - loss: 0.3139 - acc: 0.9041

In [182]:
# Evaluate Test Accuracy
scores = model.evaluate(X_test_pre, dummy_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 93.38%


In [167]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [168]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = {'C':[1,2,4,8,16],
             'gamma':[0.0625,0.125,0.25,0.5,0,1,2,4,8,16]}
clf = SVC()
clf = GridSearchCV(clf,parameters,scoring='accuracy',cv=10)
clf.fit(X_train_pre, y_train) 

AttributeError: 'GridSearchCV' object has no attribute 'best_scores'

In [169]:
label_p = clf.predict(X_test_pre)
accuracy = 100*sum(y_test==label_p)/len(y_test)

In [170]:
# Test Accuracy of 99.875%
clf.best_score_
clf.best_params_
accuracy

99.875

In [172]:
clf.best_score_

0.99468749999999995

In [174]:
clf.best_params_

{'C': 16, 'gamma': 0.25}